In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from loky import ProcessPoolExecutor
from itertools import chain

# Sur le site `pukanina.com`

In [2]:
response = requests.get("https://www.pukanina.com/faq-foire-aux-questions-infos-pratiques/")
QandA = BeautifulSoup(response.text).select(".et_pb_toggle")

questions = [elem.h5.text for elem in QandA]
answers = [elem.div.text[1:] for elem in QandA]

# Sur le site `perou.org`

In [3]:
def webscraper_decouvrir_le_perou(url):
    response = requests.get(url)
    BS1 = (BeautifulSoup(response.text)
          .select("body > h3"))
    BS2 = (BeautifulSoup(response.text)
          .select("body > blockquote"))
    n = len(BS1)
    
    return [(BS1[i].text,BS2[i].text) for i in range(n)]

def all_id_title(key):
    with ProcessPoolExecutor() as epool:
        mapped_values = epool.map(webscraper_decouvrir_le_perou, key)
    return list(chain(*mapped_values))

liste_url = ["https://perou.org/faq/trip.php",
             "https://perou.org/faq/health.php",
             "https://perou.org/faq/transports.php",
             "https://perou.org/faq/security.php",
             "https://perou.org/faq/live_peru.php"]

In [4]:
%%time
QandA = all_id_title(liste_url)

Wall time: 2.38 s


In [5]:
for q,a in QandA:
    questions.append(q.replace(" \r\n ",""))
    answers.append(a.replace('(adsbygoogle = window.adsbygoogle || []).push({});',"").replace("\r\n","").strip())

In [6]:
dico_themes = {1:"Voyage",
          2:"Administratif",
          3:"Argent",
          4:"Santé",
          5:"Communication",
          6:"Bagages",
          7:"Hébergement",
          8:"Transport",
          9:"Trek/Visite/Guide",
          10:"Vivre au Pérou",
          11:"Sécurité"}
liste_themes = [1,1,2,1,1,9,3,9,9,1,3,2,2,11,4,4,4,4,4,4,4,4,5,5,5,5,6,6,6,6,6,3,3,3,3,3,7,7,7,8,8,8,8,8,9,9,9,9,
                1,9,7,1,1,11,6,11,2,3,9,1,9,4,4,8,8,8,8,8,8,11,11,11,11,10,10,10,10,10]

themes = [dico_themes[elem] for elem in liste_themes]

In [7]:
df = pd.DataFrame(data = {'Themes':themes,'Questions': questions, 'Answers': answers})
df.head(3)

,Themes,Questions,Answers
0,Voyage,Pour et contre d'un voyage groupé en tout peti...,Nous programmons des voyages groupés à dates f...
1,Voyage,Pour et contre d'un voyage sur mesure/individu...,Avantages:\n\nun circuit individuel est modifi...
2,Administratif,Pourquoi vous n'incluez pas les commissions ba...,Nous n’incluons pas les commissions bancaires ...


In [8]:
df.tail()

,Themes,Questions,Answers
73,Vivre au Pérou,Je voudrais vivre au Pérou et/ou trouver du t...,"Si vous cherchez un travail temporaire, sans p..."
74,Vivre au Pérou,Où apprendre l'espagnol ?,"Le meilleur moyen d'apprendre l'espagnol est, ..."
75,Vivre au Pérou,Comment sont vus les Français au quotidien ?,Les Français sont généralement très bien vus a...
76,Vivre au Pérou,Quels sont les salaires pratiqués au Pérou ?,"Le salaire de base, perçu par la majorité des ..."
77,Vivre au Pérou,Où se loger à Lima et à combien ?,Le Pérou est un pays pauvre et a donc forcémen...


In [9]:
df.to_csv('../Data/Q&A.csv', sep = ';', index = False, encoding = 'utf-8', na_rep = 'NA')